In [ ]:
!pip install ultralytics

In [ ]:
import os
import cv2
import math
import csv
import threading
import pandas as pd
from pathlib import Path
from ultralytics import YOLO

# --- CONFIGURATION ---

# Model paths
MODEL1_PATH = '/content/my_saved_model_teeth.pt'
MODEL2_PATH = '/content/best2.pt'
MODEL3_PATH = '/content/my_model_teeth_height.pt'

# Input image directories
INPUT_DIR_TOP = '/content/drive/MyDrive/undistorted gear images'
INPUT_DIR_SIDE = '/content/drive/MyDrive/undistorted gear side view images'

# Output directories
OUTPUT_DIR_TOP = '/content/output_top_view'
OUTPUT_DIR_SIDE = '/content/output_side_view'
RED_MASK_DIR = '/content/output_side_red_masks'

# CSV result file
CSV_PATH_MODEL2 = '/content/gear_inspection_results_model2.csv'
CSV_PATH_MODEL3 = '/content/gear_inspection_results_model3.csv'


# Calibration and constants
PIXEL_TO_MM_TOP = 0.1976
PIXELS_PER_MM_SIDE = 116
EXPECTED_OD_MM = 40.0
OD_TOLERANCE = 1.35
TOOTH_CONFIDENCE_THRESHOLD = 0.79
THICKNESS_THRESHOLD = 0.2
MODULE = 2
EXPECTED_TEETH = 18

# Create output dirs
for d in [OUTPUT_DIR_TOP, OUTPUT_DIR_SIDE, RED_MASK_DIR]:
    os.makedirs(d, exist_ok=True)


In [ ]:
def label_teeth_defect(detected, expected):
    return detected == expected

def run_model1_teeth_validation():
    model1 = YOLO(MODEL1_PATH)
    input_images = [f for f in os.listdir(INPUT_DIR_TOP) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]

    if not input_images:
        print(f"⚠️ No images found in input folder: {INPUT_DIR_TOP}")
        return
    else:
        print(f"ℹ️ Found {len(input_images)} images in input folder for Model 1.")

    saved_non_defective_images = 0
    CLASS_NAMES = {0: "gear", 1: "tooth"}
    results_table = []

    print("=== Running Model 1: Teeth Detection ===")

    for fname in input_images:
        img_path = os.path.join(INPUT_DIR_TOP, fname)
        img = cv2.imread(img_path)

        if img is None:
            print(f"⚠️ Cannot read image {fname}, skipping.")
            continue

        results = model1(img_path, verbose=False)[0]
        detected_teeth = 0

        if results.boxes is not None and results.boxes.cls is not None:
            boxes = results.boxes
            confs = boxes.conf.cpu().numpy()
            class_ids = boxes.cls.cpu().numpy()

            print(f"📸 {fname} detections:")
            for i, (cls_id, conf) in enumerate(zip(class_ids, confs)):
                label = CLASS_NAMES.get(int(cls_id), f"class_{int(cls_id)}")
                print(f"  🔹 Detection {i+1}: Class = {label}, Confidence = {conf:.4f}")
                if int(cls_id) == 1:
                    detected_teeth += 1

        passed = label_teeth_defect(detected_teeth, EXPECTED_TEETH)

        results_table.append({
            "Image": fname,
            "Detected Teeth": detected_teeth,
            "Status": "✅ Pass" if passed else "❌ Fail"
        })

        if passed:
            print(f"✅ {fname}: Teeth OK ({detected_teeth}) - saved for Model 2")
            save_path = os.path.join(OUTPUT_DIR_TOP, fname)
            cv2.imwrite(save_path, img)
            saved_non_defective_images += 1
        else:
            print(f"❌ {fname}: Defective teeth count ({detected_teeth}) != expected ({EXPECTED_TEETH})")

    df = pd.DataFrame(results_table)
    print("\n=== Model 1: Summary Results ===")
    print(df.to_string(index=False))

    if saved_non_defective_images == 0:
        print(f"\n⚠️ No non-defective images found after Model 1 detection.")
    else:
        print(f"\n✅ Saved {saved_non_defective_images} non-defective images to: {OUTPUT_DIR_TOP}")

# Block 2
run_model1_teeth_validation()


ℹ️ Found 45 images in input folder for Model 1.
=== Running Model 1: Teeth Detection ===
📸 10.jpg detections:
  🔹 Detection 1: Class = gear, Confidence = 0.9474
  🔹 Detection 2: Class = tooth, Confidence = 0.8243
  🔹 Detection 3: Class = tooth, Confidence = 0.8212
  🔹 Detection 4: Class = tooth, Confidence = 0.8119
  🔹 Detection 5: Class = tooth, Confidence = 0.8003
  🔹 Detection 6: Class = tooth, Confidence = 0.7917
  🔹 Detection 7: Class = tooth, Confidence = 0.7815
  🔹 Detection 8: Class = tooth, Confidence = 0.7672
  🔹 Detection 9: Class = tooth, Confidence = 0.7509
  🔹 Detection 10: Class = tooth, Confidence = 0.7504
  🔹 Detection 11: Class = tooth, Confidence = 0.7503
  🔹 Detection 12: Class = tooth, Confidence = 0.7462
  🔹 Detection 13: Class = tooth, Confidence = 0.7422
  🔹 Detection 14: Class = tooth, Confidence = 0.7234
  🔹 Detection 15: Class = tooth, Confidence = 0.7032
  🔹 Detection 16: Class = tooth, Confidence = 0.7027
  🔹 Detection 17: Class = tooth, Confidence = 0.6604

In [ ]:
import os
import cv2
import pandas as pd
from ultralytics import YOLO  # Assuming you use ultralytics YOLO

def run_model2_outer_diameter_validation(
    input_folder=OUTPUT_DIR_TOP,
    annotated_folder=OUTPUT_DIR_TOP,
    circle_folder=OUTPUT_DIR_TOP,
    correctly_manufactured_folder='/content/output_correct_gears',
    log_path=CSV_PATH_MODEL2,
    pixel_to_mm=PIXEL_TO_MM_TOP,
    expected_diameter_mm=EXPECTED_OD_MM,
    tolerance_mm=OD_TOLERANCE,
    model2=None,
    error_correction_mm=0
):
    os.makedirs(annotated_folder, exist_ok=True)
    os.makedirs(circle_folder, exist_ok=True)
    os.makedirs(correctly_manufactured_folder, exist_ok=True)

    image_files = [f for f in os.listdir(input_folder) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
    if not image_files:
        print(f"⚠️ No images found in {input_folder}")
        return 0

    log_entries = []  # Will hold dicts for DataFrame rows

    saved_annotated = 0
    saved_circles = 0
    saved_correct_gears = 0

    print(f"\n=== Running Model 2: Outer Diameter Validation on {len(image_files)} images ===")

    for fname in image_files:
        img_path = os.path.join(input_folder, fname)
        img = cv2.imread(img_path)
        if img is None:
            print(f"⚠️ Cannot read image {fname}, skipping.")
            continue

        results = model2(img_path)[0]

        annotated_img = img.copy()
        is_defective = False

        boxes = getattr(results.boxes, 'xyxy', None)
        if boxes is not None:
            if hasattr(boxes, 'cpu'):
                boxes_np = boxes.cpu().numpy()
            else:
                boxes_np = boxes

            if len(boxes_np) > 0:
                box = boxes_np[0]
                x1, y1, x2, y2 = map(int, box)
                cx, cy = (x1 + x2) // 2, (y1 + y2) // 2

                radius = min((x2 - x1), (y2 - y1)) // 2
                diameter_px = radius * 2

                raw_diameter_mm = diameter_px * pixel_to_mm
                corrected_diameter_mm = raw_diameter_mm + error_correction_mm

                diff = abs(corrected_diameter_mm - expected_diameter_mm)

                if diff <= tolerance_mm:
                    status = "PASS"
                    color = (0, 255, 0)
                else:
                    status = "FAIL"
                    color = (0, 0, 255)
                    is_defective = True

                cv2.rectangle(annotated_img, (x1, y1), (x2, y2), color, 2)
                cv2.circle(annotated_img, (cx, cy), radius, color, 2)
                cv2.putText(annotated_img, f"{status} {corrected_diameter_mm:.2f}mm", (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

                # Log entry dictionary for DataFrame
                log_entries.append({
                    "Image Name": fname,
                    "Status": status,
                    "Diameter (mm)": round(corrected_diameter_mm, 2)
                })

            else:
                # No bounding box detected case
                status = "FAIL"
                log_entries.append({
                    "Image Name": fname,
                    "Status": status,
                    "Diameter (mm)": None
                })
                cv2.putText(annotated_img, "FAIL No detection", (20, 30),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)
                is_defective = True
        else:
            status = "FAIL"
            log_entries.append({
                "Image Name": fname,
                "Status": status,
                "Diameter (mm)": None
            })
            cv2.putText(annotated_img, "FAIL No detection", (20, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)
            is_defective = True

        annotated_path = os.path.join(annotated_folder, fname)
        circle_path = os.path.join(circle_folder, fname)
        cv2.imwrite(annotated_path, annotated_img)
        cv2.imwrite(circle_path, annotated_img)
        saved_annotated += 1
        saved_circles += 1

        if not is_defective:
            correct_path = os.path.join(correctly_manufactured_folder, fname)
            cv2.imwrite(correct_path, annotated_img)
            saved_correct_gears += 1

    # Save CSV with only required columns
    os.makedirs(os.path.dirname(log_path), exist_ok=True)
    df_log = pd.DataFrame(log_entries)
    df_log.to_csv(log_path, index=False)

    print(f"\n✅ Processed {saved_annotated} images with annotations.")
    print(f"✅ Saved {saved_circles} circle-annotated images to: {circle_folder}")
    print(f"✅ Saved {saved_correct_gears} correctly manufactured gears to: {correctly_manufactured_folder}")
    print(f"✅ Diameter log saved at: {log_path}")

    return saved_correct_gears


# Usage example:
model2 = YOLO(MODEL2_PATH)

correct_count = run_model2_outer_diameter_validation(
    model2=model2
)
print(f"Total correctly manufactured gears saved: {correct_count}")



=== Running Model 2: Outer Diameter Validation on 45 images ===

image 1/1 /content/output_top_view/38.jpg: 384x640 1 inner circle, 1 outer circle, 9.0ms
Speed: 2.8ms preprocess, 9.0ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

image 1/1 /content/output_top_view/47.jpg: 384x640 1 inner circle, 1 outer circle, 11.0ms
Speed: 2.9ms preprocess, 11.0ms inference, 1.9ms postprocess per image at shape (1, 3, 384, 640)

image 1/1 /content/output_top_view/42.jpg: 384x640 1 inner circle, 1 outer circle, 13.4ms
Speed: 2.9ms preprocess, 13.4ms inference, 2.3ms postprocess per image at shape (1, 3, 384, 640)

image 1/1 /content/output_top_view/18.jpg: 384x640 1 inner circle, 1 outer circle, 9.8ms
Speed: 3.1ms preprocess, 9.8ms inference, 1.9ms postprocess per image at shape (1, 3, 384, 640)

image 1/1 /content/output_top_view/31.jpg: 384x640 1 inner circle, 1 outer circle, 16.3ms
Speed: 3.1ms preprocess, 16.3ms inference, 1.9ms postprocess per image at shape (1, 3, 384, 640)

In [ ]:
import os
import cv2
import numpy as np
import csv
import pandas as pd
from ultralytics import YOLO

def run_model3_teeth_height_validation(
    input_folder,
    output_folder,
    log_path,
    model3,
    pixel_to_mm,
    conf_threshold=0.79,
    target_height_mm=20,
    thickness_correction=0.2,
):
    os.makedirs(output_folder, exist_ok=True)
    os.makedirs(os.path.dirname(log_path), exist_ok=True)

    image_files = [f for f in os.listdir(input_folder) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
    if not image_files:
        print(f"⚠️ No images found in {input_folder}")
        return 0, pd.DataFrame()

    log_lines = []
    saved_images = 0

    for fname in image_files:
        img_path = os.path.join(input_folder, fname)
        img = cv2.imread(img_path)
        if img is None:
            print(f"⚠️ Cannot read image {fname}, skipping.")
            continue

        results = model3(img)

        boxes = results[0].boxes.xyxy.cpu().numpy() if results[0].boxes else np.array([])
        scores = results[0].boxes.conf.cpu().numpy() if results[0].boxes else np.array([])

        keep = scores >= conf_threshold
        boxes = boxes[keep]
        scores = scores[keep]

        annotated_img = img.copy()

        for box, score in zip(boxes, scores):
            x1, y1, x2, y2 = map(int, box)
            cv2.rectangle(annotated_img, (x1, y1), (x2, y2), (0, 0, 255), 2)
            label = f"{score:.2f}"
            (w, h), _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 2)
            cv2.rectangle(annotated_img, (x1, y1 - h - 4), (x1 + w, y1), (0, 0, 255), -1)
            cv2.putText(annotated_img, label, (x1, y1 - 2), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 0), 2)

        hsv = cv2.cvtColor(annotated_img, cv2.COLOR_BGR2HSV)
        lower_red1 = np.array([0, 50, 50])
        upper_red1 = np.array([10, 255, 255])
        lower_red2 = np.array([160, 50, 50])
        upper_red2 = np.array([180, 255, 255])
        mask1 = cv2.inRange(hsv, lower_red1, upper_red1)
        mask2 = cv2.inRange(hsv, lower_red2, upper_red2)
        mask = cv2.bitwise_or(mask1, mask2)

        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        heights_pixels = [cv2.boundingRect(cnt)[3] for cnt in contours]

        if not heights_pixels:
            log_lines.append(f"{fname},No red bounding boxes detected,,")
            corrected_thickness = None
        else:
            heights_mm = [h * pixel_to_mm for h in heights_pixels]
            diffs = [abs(h - target_height_mm) for h in heights_mm]
            min_idx = diffs.index(min(diffs))
            closest_height_mm = heights_mm[min_idx]
            closest_height_px = heights_pixels[min_idx]

            corrected_thickness = closest_height_mm - thickness_correction
            log_lines.append(f"{fname},{closest_height_px},{closest_height_mm:.3f},{corrected_thickness:.3f}")

        output_path = os.path.join(output_folder, fname)
        cv2.imwrite(output_path, annotated_img)
        saved_images += 1

    with open(log_path, "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(["Image Name", "Closest Height (pixels)", "Closest Height (mm)", "Corrected Thickness (mm)"])
        for line in log_lines:
            writer.writerow(line.split(","))

    # Return count and the pandas DataFrame for the CSV log
    df_log = pd.read_csv(log_path)
    return saved_images, df_log


# Usage example (make sure MODEL3_PATH, INPUT_DIR_SIDE, OUTPUT_DIR_SIDE, CSV_PATH_MODEL3 are defined):
model3 = YOLO(MODEL3_PATH)

count, df = run_model3_teeth_height_validation(
    input_folder=INPUT_DIR_SIDE,
    output_folder=OUTPUT_DIR_SIDE,
    log_path=CSV_PATH_MODEL3,
    model3=model3,
    pixel_to_mm=2/232,
    conf_threshold=0.79,
    target_height_mm=20,
    thickness_correction=0.2
)
print(f"Processed {count} images.")
print(df.head())  # print first few rows of DataFrame



0: 384x640 8 teeths, 17.1ms
Speed: 3.8ms preprocess, 17.1ms inference, 1.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 teeths, 14.9ms
Speed: 3.7ms preprocess, 14.9ms inference, 2.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 teeths, 10.6ms
Speed: 3.5ms preprocess, 10.6ms inference, 1.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 teeths, 10.7ms
Speed: 3.5ms preprocess, 10.7ms inference, 1.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 teeths, 10.6ms
Speed: 3.7ms preprocess, 10.6ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 9 teeths, 10.8ms
Speed: 3.8ms preprocess, 10.8ms inference, 2.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 teeths, 11.3ms
Speed: 3.7ms preprocess, 11.3ms inference, 1.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 teeths, 10.6ms
Speed: 3.6ms preprocess, 10.6ms inference, 1.9ms postprocess per image at shape (1, 3, 38

In [ ]:
import threading
import pandas as pd
import os

# CSV paths from your config
CSV_PATH_MODEL3 = '/content/gear_inspection_results_model3.csv'
MERGED_CSV = '/content/gear_inspection_results_merged.csv'

# Static Model 2 CSV filenames (no timestamp)
RAW_CSV_MODEL2 = '/content/gear_inspection_results_model2.csv'
FINAL_CSV_MODEL2 = '/content/gear_inspection_results_model2.csv'

results = {
    "model2_count": None,
    "model3_count": None,
}

def thread_model2():
    print("[Model 2] Starting processing...")

    # Run model 2 and save raw CSV at RAW_CSV_MODEL2
    count = run_model2_outer_diameter_validation(
        input_folder=INPUT_DIR_TOP,
        circle_folder=OUTPUT_DIR_TOP,
        model2=model2,
        pixel_to_mm=PIXEL_TO_MM_TOP,
        log_path=RAW_CSV_MODEL2,
    )
    print(f"[Model 2] Raw CSV saved at: {RAW_CSV_MODEL2}")

    if not os.path.exists(RAW_CSV_MODEL2):
        raise FileNotFoundError(f"[Model 2] File {RAW_CSV_MODEL2} not found.")

    # Load raw CSV and process columns
    df2 = pd.read_csv(RAW_CSV_MODEL2)

    # Find diameter column starting with 'Corrected='
    diameter_col = next((col for col in df2.columns if col.startswith('Corrected=')), None)

    if diameter_col is None:
        raise ValueError("[Model 2] Diameter column with 'Corrected=' prefix not found in CSV.")

    # Convert that diameter column values to float mm (strip 'mm')
    df2['Outer Diameter (mm)'] = pd.to_numeric(df2[diameter_col].astype(str).str.replace('mm', '').str.strip(), errors='coerce')

    # Check required columns
    for c in ['Image Name', 'Status']:
        if c not in df2.columns:
            raise ValueError(f"[Model 2] Required column '{c}' missing from CSV.")

    # Save filtered columns to final CSV
    df2[['Image Name', 'Status', 'Outer Diameter (mm)']].to_csv(FINAL_CSV_MODEL2, index=False)
    print(f"[Model 2] Final CSV saved at: {FINAL_CSV_MODEL2}")

    results["model2_count"] = count

def thread_model3():
    print("[Model 3] Starting processing...")
    count = run_model3_teeth_height_validation(
        input_folder=INPUT_DIR_SIDE,
        output_folder=OUTPUT_DIR_SIDE,
        log_path=CSV_PATH_MODEL3,
        model3=model3,
        pixel_to_mm=2/232,
        conf_threshold=0.79,
        target_height_mm=20,
        thickness_correction=0.2,
    )
    print(f"[Model 3] Finished processing with {count} images.")
    results["model3_count"] = count

# Start threads for both models
t2 = threading.Thread(target=thread_model2)
t3 = threading.Thread(target=thread_model3)

t2.start()
t3.start()

t2.join()
t3.join()

print(f"Model 2 processed {results['model2_count']} images.")
print(f"Model 3 processed {results['model3_count']} images.")

# Load processed CSVs
df2 = pd.read_csv(FINAL_CSV_MODEL2)
df3 = pd.read_csv(CSV_PATH_MODEL3)

df2['source_model'] = 'model2'
df3['source_model'] = 'model3'

# Merge on 'Image Name'
new_merged = pd.merge(df2, df3, on='Image Name', how='outer', suffixes=('_model2', '_model3'))

# Merge with existing CSV if exists
if os.path.exists(MERGED_CSV):
    existing_df = pd.read_csv(MERGED_CSV)
    combined_df = pd.merge(existing_df, new_merged, on='Image Name', how='outer', suffixes=('_old', '_new'))

    # Resolve old/new columns preferring new data when available
    for col in combined_df.columns:
        if col.endswith('_old'):
            base_col = col[:-4]
            new_col = base_col + '_new'
            if new_col in combined_df.columns:
                combined_df[base_col] = combined_df[new_col].combine_first(combined_df[col])
                combined_df.drop([col, new_col], axis=1, inplace=True)
            else:
                combined_df.rename(columns={col: base_col}, inplace=True)
else:
    combined_df = new_merged

# Save merged results
combined_df.to_csv(MERGED_CSV, index=False)
print(f"Merged results saved to: {MERGED_CSV}")


[Model 2] Starting processing...

=== Running Model 2: Outer Diameter Validation on 45 images ===
[Model 3] Starting processing...


image 1/1 /content/drive/MyDrive/undistorted gear images/10.jpg: 384x640 1 inner circle, 1 outer circle, 65.5ms
Speed: 3.6ms preprocess, 65.5ms inference, 2.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 teeths, 69.0ms
Speed: 3.9ms preprocess, 69.0ms inference, 4.8ms postprocess per image at shape (1, 3, 384, 640)
image 1/1 /content/drive/MyDrive/undistorted gear images/11.jpg: 384x640 1 inner circle, 1 outer circle, 14.9ms
Speed: 3.1ms preprocess, 14.9ms inference, 3.4ms postprocess per image at shape (1, 3, 384, 640)

image 1/1 /content/drive/MyDrive/undistorted gear images/12.jpg: 384x640 1 inner circle, 1 outer circle, 9.8ms
Speed: 3.1ms preprocess, 9.8ms inference, 2.1ms postprocess per image at shape (1, 3, 384, 640)


image 1/1 /content/drive/MyDrive/undistorted gear images/13.jpg: 384x640 1 inner circle, 1 outer circle, 18.4ms
S

Exception in thread Thread-31 (thread_model2):
Traceback (most recent call last):
  File "/usr/lib/python3.11/threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.11/threading.py", line 982, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-21-ffbe73346acb>", line 41, in thread_model2
ValueError: [Model 2] Diameter column with 'Corrected=' prefix not found in CSV.



✅ Processed 45 images with annotations.
✅ Saved 45 circle-annotated images to: /content/output_top_view
✅ Saved 45 correctly manufactured gears to: /content/output_correct_gears
✅ Diameter log saved at: /content/gear_inspection_results_model2.csv
[Model 2] Raw CSV saved at: /content/gear_inspection_results_model2.csv

0: 384x640 9 teeths, 10.5ms
Speed: 3.2ms preprocess, 10.5ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 teeths, 11.1ms
Speed: 3.5ms preprocess, 11.1ms inference, 2.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 teeths, 10.6ms
Speed: 3.6ms preprocess, 10.6ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 9 teeths, 10.5ms
Speed: 3.2ms preprocess, 10.5ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 teeths, 10.6ms
Speed: 3.1ms preprocess, 10.6ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 teeths, 10.6ms
Speed: 3.0ms preproce

In [ ]:
!pip install gspread pandas google-auth

In [ ]:
import pandas as pd
import gspread
from google.oauth2.service_account import Credentials

# Your Google Sheet ID and credentials file path
GOOGLE_SHEET_ID = '1nbvkXbZ4KdbWb9DhygcD-pY_3Y-y_Bd0Ts0gTQi3gA8'
CREDENTIALS_JSON_PATH = '/content/credentials.json'

# Set the path to your merged CSV file here:
MERGED_CSV_PATH = '/content/gear_inspection_results_merged.csv'  # <-- change this to your CSV file path

SCOPES = ['https://www.googleapis.com/auth/spreadsheets']

def upload_csv_to_google_sheets(csv_path):
    df = pd.read_csv(csv_path)
    creds = Credentials.from_service_account_file(CREDENTIALS_JSON_PATH, scopes=SCOPES)
    client = gspread.authorize(creds)
    sheet = client.open_by_key(GOOGLE_SHEET_ID).sheet1
    sheet.clear()
    sheet.update([df.columns.values.tolist()] + df.values.tolist())
    print("✅ CSV uploaded successfully!")

upload_csv_to_google_sheets(MERGED_CSV_PATH)


✅ CSV uploaded successfully!
